In [4]:

from dotenv import load_dotenv
import os
from azure.identity import AzureDeveloperCliCredential, DefaultAzureCredential
import textwrap
import json

load_dotenv()

knowledge_source_name = "alta-knowledge-source"
knowledge_base_name = "alta-knowledge-agent"
azure_search_credential = AzureDeveloperCliCredential()
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_embedding_model = os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large")
aoai_embedding_deployment = os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large")
aoai_gpt_model = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-5-mini")
aoai_gpt_deployment = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-5-mini")

instructions = """
A Q&A agent that can answer questions about the Alta Backup product.
If you don't have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

In [5]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseRetrievalRequest, KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, SearchIndexKnowledgeSourceParams
from azure.search.documents.indexes.models import KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeRetrievalOutputMode, KnowledgeRetrievalLowReasoningEffort

agent_client = KnowledgeBaseRetrievalClient(endpoint=search_endpoint, knowledge_base_name=knowledge_base_name, credential=azure_search_credential)
query_1 = """
    how to configure backup policy for oracle on windows server?
    """

messages.append({
    "role": "user",
    "content": query_1
})

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(
            role=m["role"],
            content=[KnowledgeBaseMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            include_references=True,
            include_reference_source_data=True,
            always_query_source=True
        )
    ],
    include_activity=True,
    retrieval_reasoning_effort=KnowledgeRetrievalLowReasoningEffort
)

result = agent_client.retrieve(retrieval_request=req)
print(f"Retrieved content from '{knowledge_base_name}' successfully.")

Retrieved content from 'alta-knowledge-agent' successfully.


In [7]:
response_contents = []
activity_contents = []
references_contents = []

In [8]:
import json

# Build simple string values for response_content, activity_content, and references_content

# Responses -> Concatenate text/value fields from all response contents
response_parts = []
for resp in result.response:
    for content in resp.content:
        response_parts.append(content.text)
response_content = "\n\n".join(response_parts) if response_parts else "No response found on 'result'"

response_contents.append(response_content)

# Print the three string values
print("response_content:\n", response_content, "\n")

response_content:
 To configure a backup policy for an Oracle database on a Windows Server using NetBackup, follow these steps:

1. In the Backup Selections tab of the policy, add the mounted path on the NFS client server and the exported path of the Universal Share using the directive format: BACKUP X USING Y. For example, if the Universal Share is mounted on a Windows system at C:\\mounted\\MOUNTED, use the path format /C:/mounted/MOUNTED for the BACKUP path.

2. You can add multiple shares in a policy. If you want to group several shares into one backup job, use the NEW_STREAM directive for multistream backup.

3. For Oracle agent-specific configuration, set the DB_SCRIPT_PATH using the nbsetconfig command. On Windows, this might look like:
   C:\\Program Files\\Veritas\\NetBackup\\bin>nbsetconfig
   nbsetconfig> DB_SCRIPT_PATH=c:\\db_scripts
   nbsetconfig> DB_SCRIPT_PATH=e:\\oracle\\fullbackup\\full_rman.sh
   nbsetconfig>

4. Configure the RMAN_OUTPUT_DIR to specify the directory

In [10]:
messages.append({
    "role": "assistant",
    "content": response_content
})

In [11]:
# Activity -> JSON string of activity as list of dicts
if result.activity:
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 1768,
    "input_tokens": 1459,
    "output_tokens": 55
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 923,
    "knowledge_source_name": "alta-knowledge-source",
    "query_time": "2025-12-04T08:07:19.672Z",
    "count": 3,
    "search_index_arguments": {
      "search": "configure backup policy for Oracle database on Windows Server",
      "source_data_fields": [
        {
          "name": "section_title"
        },
        {
          "name": "para"
        },
        {
          "name": "category"
        },
        {
          "name": "topics"
        },
        {
          "name": "id"
        },
        {
          "name": "part_id"
        },
        {
          "name": "summary"
        }
      ],
      "search_fields": [],
      "semantic_configuration_name": "aml-semantic-config"
    }
  },
  {
    "id": 2,
    "type": "agenticReasoning",
    "reasoning_tokens": 2080

In [12]:
# References -> JSON string of references as list of dicts
if result.references:
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

references_content:
 [
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 1,
    "source_data": {
      "id": "442c2a19-6ea6-462d-acc2-075dde0bd298",
      "section_title": "Backup Selections tab",
      "para": "\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tClick OK to close and save the policy.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tClick Cancel to close the policy without saving any additions or changes.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t(Optional) Add the NEW_STREAM directive if you require multistream backup.\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\tAdd the mounted path on the NFS client server and the exported path of the Universal Share.\n\t\t\t\t\t\t\t\t\t\tBACKUP /\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tEnter the BACKUP X USING Y directive in the Backup Selections tab. For example: BACKUP /mou

#### Run Evaluation

In [16]:
query_2 = "what about on linux servers?"

messages.append({
    "role": "user",
    "content": query_2
})

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(
            role=m["role"],
            content=[KnowledgeBaseMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            include_references=True,
            include_reference_source_data=True,
            always_query_source=True
        )
    ],
    include_activity=True,
    retrieval_reasoning_effort=KnowledgeRetrievalLowReasoningEffort
)

result = agent_client.retrieve(retrieval_request=req)
print(f"Retrieved content from '{knowledge_base_name}' successfully.")

Retrieved content from 'alta-knowledge-agent' successfully.


In [17]:
import json

# Build simple string values for response_content, activity_content, and references_content

# Responses -> Concatenate text/value fields from all response contents
response_parts = []
for resp in result.response:
    for content in resp.content:
        response_parts.append(content.text)
response_content = "\n\n".join(response_parts) if response_parts else "No response found on 'result'"

response_contents.append(response_content)

# Print the three string values
print("response_content:\n", response_content, "\n")

response_content:
 To configure a backup policy for Oracle on Linux servers using NetBackup, follow these detailed steps:

1. **Add Backup Selections to the Policy:**
   - In the Backup Selections tab, specify the mounted path on the NFS client server and the exported path of the Universal Share using the directive format: `BACKUP X USING Y`. For example: `BACKUP /mounted/MOUNTED USING /shares/EXPORTED`.
   - If the Universal Share is mounted on a Windows system, use the format `/C:/mounted/MOUNTED` for the BACKUP path, but for Linux servers, use the standard UNIX path format.
   - You can add multiple shares in a policy. To group several shares into one backup job, use the `NEW_STREAM` directive for multistream backup if required.

2. **Configure Oracle Agent Script Paths:**
   - Use the appropriate script path for the Oracle agent on Linux. For example, set the `DB_SCRIPT_PATH` using the `nbsetconfig` command:
     ```
     [root@client26 bin]# ./nbsetconfig
     nbsetconfig>DB_SCRIP

In [18]:
# Activity -> JSON string of activity as list of dicts
if result.activity:
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
else:
    activity_content = "No activity found on 'result'"
    
activity_contents.append(activity_content)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 2206,
    "input_tokens": 2259,
    "output_tokens": 97
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 779,
    "knowledge_source_name": "alta-knowledge-source",
    "query_time": "2025-12-04T08:16:13.572Z",
    "count": 6,
    "search_index_arguments": {
      "search": "configure backup policy for oracle on linux servers",
      "source_data_fields": [
        {
          "name": "section_title"
        },
        {
          "name": "para"
        },
        {
          "name": "category"
        },
        {
          "name": "topics"
        },
        {
          "name": "id"
        },
        {
          "name": "part_id"
        },
        {
          "name": "summary"
        }
      ],
      "search_fields": [],
      "semantic_configuration_name": "aml-semantic-config"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 767,
    "knowledge_sourc

In [19]:
# References -> JSON string of references as list of dicts
if result.references:
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
else:
    references_content = "No references found on 'result'"
    
references_contents.append(references_content)
print("references_content:\n", references_content)

references_content:
 [
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 1,
    "source_data": {
      "id": "442c2a19-6ea6-462d-acc2-075dde0bd298",
      "section_title": "Backup Selections tab",
      "para": "\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tClick OK to close and save the policy.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tClick Cancel to close the policy without saving any additions or changes.\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t(Optional) Add the NEW_STREAM directive if you require multistream backup.\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\tAdd the mounted path on the NFS client server and the exported path of the Universal Share.\n\t\t\t\t\t\t\t\t\t\tBACKUP /\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tEnter the BACKUP X USING Y directive in the Backup Selections tab. For example: BACKUP /mou

In [20]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

foundry_endpoint = os.environ["AZURE_AI_AGENT_ENDPOINT"]
aoai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

# Run the evaluation
from azure.ai.evaluation import AzureOpenAIModelConfiguration, GroundednessEvaluator, RelevanceEvaluator, evaluate
import json

evaluation_data = []
print("Preparing evaluation data...")
for q, r, g in zip([query_1, query_2], references_contents, response_contents):
    evaluation_data.append({
        "query": q,
        "response": g,
        "context": r,
    })

filename = "evaluation_data.jsonl"

with open(filename, "w") as f:
    for item in evaluation_data:
        f.write(json.dumps(item) + "\n")

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
    azure_deployment=aoai_gpt_model
)

# RAG triad metrics
groundedness = GroundednessEvaluator(model_config=model_config)
relevance = RelevanceEvaluator(model_config=model_config)

print("Starting evaluation...")
result = evaluate(
    data=filename,
    evaluators={
        "groundedness": groundedness,
        "relevance": relevance,
    },
    azure_ai_project=foundry_endpoint,
)

print("Evaluation complete.")
studio_url = result.get("studio_url")
print("For more information, go to the Foundry portal.") if studio_url else None

Preparing evaluation data...
Starting evaluation...
2025-12-04 00:18:11 -0800   26408 execution.bulk     INFO     Finished 1 / 2 lines.
2025-12-04 00:18:11 -0800   26408 execution.bulk     INFO     Average execution time for completed lines: 9.04 seconds. Estimated time for incomplete lines: 9.04 seconds.
2025-12-04 00:18:11 -0800   26408 execution.bulk     INFO     Finished 2 / 2 lines.
2025-12-04 00:18:11 -0800   26408 execution.bulk     INFO     Average execution time for completed lines: 4.81 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "relevance_20251204_081801_969713"
Run status: "Completed"
Start time: "2025-12-04 08:18:01.969713+00:00"
Duration: "0:00:10.397359"

2025-12-04 00:18:12 -0800   11900 execution.bulk     INFO     Finished 1 / 2 lines.
2025-12-04 00:18:12 -0800   11900 execution.bulk     INFO     Average execution time for completed lines: 10.56 seconds. Estimated time for incomplete lines: 10.56 seconds.
2025-12-0

Aggregated metrics for evaluator is not a dictionary will not be logged as metrics
Aggregated metrics for evaluator is not a dictionary will not be logged as metrics


======= Run Summary =======

Run name: "groundedness_20251204_081801_966287"
Run status: "Completed"
Start time: "2025-12-04 08:18:01.966287+00:00"
Duration: "0:00:11.461071"

======= Combined Run Summary (Per Evaluator) =======

{
    "groundedness": {
        "status": "Completed",
        "duration": "0:00:11.461071",
        "completed_lines": 2,
        "failed_lines": 0,
        "log_path": null,
        "error_message": null,
        "error_code": null
    },
    "relevance": {
        "status": "Completed",
        "duration": "0:00:10.397359",
        "completed_lines": 2,
        "failed_lines": 0,
        "log_path": null,
        "error_message": null,
        "error_code": null
    }
}


Evaluation complete.
For more information, go to the Foundry portal.
